In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------



In [38]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)

In [29]:
# tb_pacientes = TbPacientes()
# tb_pacientes.update()
# casost = tb_pacientes.get()
casost = pd.read_pickle('tb_pacientes.pkl')#tb_pacientes.get()
casost['nome'] = casost['nome'].apply(lambda nome: ''.join(filter(lambda x: ('A' <= x <= 'Z') or ( x == ' '), nome)))
casost = casost.sort_values(['dt_notificacao','nome','idade'])
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
print(f"casost len: {len(casost)}\n")
casost.groupby('obito')[['sexo']].count()

casost len: 1170632

casost len: 1170632



,sexo
obito,
NAO,1141455
SIM,29177


In [30]:
casost

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito
8951,8838,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-03 00:00:00,Notificações,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,47,NaT,NaT,48,46,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
11016,10932,410690,410690,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,43,NaT,NaT,44,42,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN
11017,10933,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,15,NaT,NaT,16,14,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
11550,11470,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,NaN,07/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN
11572,11492,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,NaN,08/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668050,1257727,411580,411580,ZULMA ARLETE DA SILVEIRA,F,63,MEDIANEIRA,MEDIANEIRA,27,2021-05-17,2021-06-17,2021-05-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2021-05-20 00:00:00,Notifica COVID-19,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,NaT,2021-06-17,64,62,ZULMAARLETEDASILVEIRA63MEDIANEIRA,ZULMAARLETEDASILVEIRA62MEDIANEIRA,ZULMAARLETEDASILVEIRA64MEDIANEIRA,ZULMAARLETEDASILVEIRA63MEDIANEIRA,ZULMAARLETEDASILVEIRA62MEDIANEIRA,ZULMAARLETEDASILVEIRA64MEDIANEIRA,ZULMAARLETEDASILVEIRA17052021,NaN
672082,1261774,412570,412570,ZULMA BEATRI

In [31]:
casost.groupby('status')[['sexo']].count()

,sexo
status,
Recuperado,824519
recuperado,2


In [42]:
casosct = casost.copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

In [43]:
casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_atualizacao']

In [44]:
casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'dt_obito']
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_com_recuperado'] 

In [45]:
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,824521
2,29177
3,316934


In [56]:
casosct['uf_resid'] = 'PR'
casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)

casosct.loc[casosct['ibge_res_pr']==999999,'uf_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge_res_pr']==999999].drop(columns=['ibge_res_pr']),municipios[['ibge','mun_resid','uf']].rename(columns={'uf':'uf_resid','ibge':'ibge_res_pr'}), on=['mun_resid','uf_resid'], how='left')
casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'ibge_res_pr'] = 999999
casos_fora['ibge_res_pr'] = casos_fora['ibge_res_pr'].astype(int)
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge_res_pr'] = casosct['ibge_res_pr'].astype(int)
casosct['idade'] = casosct['idade'].astype(int)

casosct.loc[casosct['uf_resid']!='PR']

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito,data_evolucao,data_com_evolucao,uf_resid
5,13673,355030,410690,ITALO FERNANDO TROMBINI FILHO,M,28.0,SAO PAULO,CURITIBA,Genoprimer,2020-03-16,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-06-17 00:00:00,SMS,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,28.0,NaT,2020-08-05,29.0,27.0,ITALOFERNANDOTROMBINIFILHO28SAOPAULOSP,ITALOFERNANDOTROMBINIFILHO27SAOPAULOSP,ITALOFERNANDOTROMBINIFILHO29SAOPAULOSP,ITALOFERNANDOTROMBINIFILHO28CURITIBA,ITALOFERNANDOTROMBINIFILHO27CURITIBA,ITALOFERNANDOTROMBINIFILHO29CURITIBA,ITALOFERNANDOTROMBINIFILHO16032020,NaN,2020-06-17 00:00:00,2020-08-05,SP
63,19531,530010,410690,LUCIANO PUCHALSKI,M,59.0,BRASILIA,CURITIBA,LACEN,2020-03-23,2020-03-24,2020-03-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,23.0,59.0,NaT,NaT,60.0,58.0,LUCIANOPUCHALSKI59BRASILIADF,LUCIANOPUCHALSKI58BRASILIADF,LUCIANOPUCHALSKI60BRASILIADF,LUCIANOPUCHALSKI59CURITIBA,LUCIANOPUCHALSKI58CURITIBA,LUCIANOPUCHALSKI60CURITIBA,LUCIANOPUCHALSKI23032020,NaN,NaT,NaT,DF
77,15204,355030,410370,JOAO HENRIQUE FONZAR BEGNINI,M,37.0,SAO PAULO,CAMBE,LACEN,2020-03-25,2020-03-25,2020-03-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,23.0,37.0,NaT,NaT,38.0,36.0,JOAOHENRIQUEFONZARBEGNINI37SAOPAULOSP,JOAOHENRIQUEFONZARBEGNINI36SAOPAULOSP,JOAOHENRIQUEFONZARBEGNINI38SAOPAULOSP,JOAOHENRIQUEFONZARBEGNINI37CAMBE,JOAOHENRIQUEFONZARBEGNINI36CAMBE,JOAOHENRIQUEFONZARBEGNINI38CAMBE,JOAOHENRIQUEFONZARBEGNINI25032020,NaN,NaT,NaT,SP
175,19629,330330,411435,LUCINEIA ALVES DE ANHAIA,F,38.0,NITEROI,MANFRINOPOLIS,LACEN,2020-03-31,2020-03-31,2020-03-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,23.0,38.0,NaT,NaT,39.0,37.0,LUCINEIAALVESDEANHAIA38NITEROIRJ,LUCINEIAALVESDEANHAIA37NITEROIRJ,LUCINEIAALVESDEANHAIA39NITEROIRJ,LUCINEIAALVESDEANHAIA38MANFRINOPOLIS,LUCINEIAALVESDEANHAIA37MANFRINOPOLIS,LUCINEIAALVESDEANHAIA39MANFRINOPOLIS,LUCINEIAALVESDEANHAIA31032020,NaN,NaT,NaT,RJ
218,25622,355030,410690,PAULO FRANCISCO NETO,M,25.0,SAO PAULO,CURITIBA,LACEN,2020-03-31,2020-04-01,2020-03-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-03-19 00:00:00,SMS,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,25.0,NaT,2020-08-05,26.0,24.0,PAULOFRANCISCONETO25SAOPAULOSP,PAULOFRANCISCONETO24SAOPAULOSP,PAULOFRANCISCONETO26SAOPAULOSP,PAULOFRANCISCONETO25CURITIBA,PAULOFRANCISCONETO24CURITIBA,PAULOFRANCISCONETO26CURITIBA,PAULOFRANCISCONETO31032020,NaN,2020-03-19 00:00:00,2020-08-05,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170199,1255819,421880,410940,VALDEMAR PINTO DE CRISTO,M,52.0,TURVO,GUARAPUAVA,1,2021-06-10,2021-06-17,2021-06-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,52.0,NaT,NaT,53.0,51.0,VALDEMARPINTODECRISTO52TURVOSC,VALDEMARPINTODECRISTO51TURVOSC,VALDEMARPINTODECRISTO53TURVOSC,VALDEMARPINTODECRISTO52GUARAPUAVA,VALDEMARPINTODECRISTO51GUARAPUAVA,VALDEMARPINTODECRISTO53GUARAPUAVA,VALDEMARPINTODECRISTO10062021,NaN,NaT,NaT,SC
1170235,1260149,350070,411990,VALDIR APARECIDO DE OLIVEIRA,M,47.0,AGUDOS,PONTA GROSSA,167,2021-05-31,2021-06-17,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [57]:
casosct.groupby(['evolucao'])[['sexo']].count()

,sexo
evolucao,
1.0,824521
2.0,29177
3.0,316934


In [58]:
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge_res_pr']
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(int)
casosct['comunicacao'] = casosct['dt_notificacao']
casosct['is'] = casosct['dt_inicio_sintomas']

casosct = casosct[['identificacao', 'id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao', 'data_com_evolucao']]
casosct

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47.0,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43.0,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15.0,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26.0,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26.0,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170627,1257727,-1,PR,411580,411580,ZULMA ARLETE DA SILVEIRA,F,63.0,27,2021-05-17,2021-06-17,2021-05-11 00:00:00,1.0,2021-05-20 00:00:00,2021-06-17
1170628,1261774,-1,PR,412570,412570,ZULMA BEATRIZ LOPEZ DE FERREIRA,F,49.0,27,2021-06-15,2021-06-17,2021-06-10 00:00:00,3.0,NaT,NaT
1170629,1259702,-1,PR,410430,410430,ZULMIR LUIZ PASINI,M,76.0,1,2021-06-02,2021-06-17,2021-05-30 00:00:00,1.0,2021-06-14 00:00:00,2021-06-17
1170630,1258691,-1,PR,411300,410550,ZULMIRA APARECIDA MOREZZI GUIETI,F,69.0,167,2021-06-15,2021-06-17,2021-06-13 00:00:00,3.0,NaT,NaT


In [59]:
casos_confirmados = casosct.copy()
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47.0,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43.0,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15.0,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26.0,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26.0,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170627,1257727,-1,PR,411580,411580,ZULMA ARLETE DA SILVEIRA,F,63.0,27,2021-05-17,2021-06-17,2021-05-11 00:00:00,1.0,2021-05-20 00:00:00,2021-06-17
1170628,1261774,-1,PR,412570,412570,ZULMA BEATRIZ LOPEZ DE FERREIRA,F,49.0,27,2021-06-15,2021-06-17,2021-06-10 00:00:00,3.0,NaT,NaT
1170629,1259702,-1,PR,410430,410430,ZULMIR LUIZ PASINI,M,76.0,1,2021-06-02,2021-06-17,2021-05-30 00:00:00,1.0,2021-06-14 00:00:00,2021-06-17
1170630,1258691,-1,PR,411300,410550,ZULMIRA APARECIDA MOREZZI GUIETI,F,69.0,167,2021-06-15,2021-06-17,2021-06-13 00:00:00,3.0,NaT,NaT


In [60]:
casos_confirmados.dtypes

identificacao                 int64
id_notifica                   int64
uf_resid                     object
ibge_resid                    int64
ibge_atend                    int64
nome                         object
sexo                         object
idade                       float64
laboratorio                  object
dt_diag              datetime64[ns]
comunicacao          datetime64[ns]
is                           object
evolucao                    float64
data_evolucao                object
data_com_evolucao    datetime64[ns]
dtype: object

In [61]:
casos_confirmados.loc[casos_confirmados['is'].notna(),'is'] = casos_confirmados.loc[casos_confirmados['is'].notna(),'is'].apply(str).str[:10]
casos_confirmados['is'] = pd.to_datetime(casos_confirmados['is'],format='%d/%m/%Y',errors='ignore')
casos_confirmados['is'] = pd.to_datetime(casos_confirmados['is'],format='%Y-%m-%d',errors='coerce')
casos_confirmados['is']

0                NaT
1         2020-03-07
2         2020-03-08
3         2020-03-11
4         2020-03-15
             ...    
1170627   2021-05-11
1170628   2021-06-10
1170629   2021-05-30
1170630   2021-06-13
1170631   2021-06-06
Name: is, Length: 1170632, dtype: datetime64[ns]

In [62]:
casos_confirmados.loc[casos_confirmados['data_evolucao'].notna(),'data_evolucao'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].notna(),'data_evolucao'].apply(str).str[:10]
casos_confirmados['data_evolucao'] = pd.to_datetime(casos_confirmados['data_evolucao'],format='%d/%m/%Y',errors='ignore')
casos_confirmados['data_evolucao'] = pd.to_datetime(casos_confirmados['data_evolucao'],format='%Y-%m-%d',errors='coerce')
casos_confirmados['data_evolucao']

0         2020-04-03
1                NaT
2                NaT
3                NaT
4                NaT
             ...    
1170627   2021-05-20
1170628          NaT
1170629   2021-06-14
1170630          NaT
1170631          NaT
Name: data_evolucao, Length: 1170632, dtype: datetime64[ns]

In [63]:
casos_confirmados['evolucao'] = casos_confirmados['evolucao'].astype(int)
casos_confirmados.dtypes

identificacao                 int64
id_notifica                   int64
uf_resid                     object
ibge_resid                    int64
ibge_atend                    int64
nome                         object
sexo                         object
idade                       float64
laboratorio                  object
dt_diag              datetime64[ns]
comunicacao          datetime64[ns]
is                   datetime64[ns]
evolucao                      int64
data_evolucao        datetime64[ns]
data_com_evolucao    datetime64[ns]
dtype: object

In [64]:
casos_confirmados['hash'] = ( casos_confirmados['nome'].apply(normalize_hash) +
               casos_confirmados['idade'].astype(int).astype(str) +
               casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_less'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x-1))) +
                    casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_more'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x+1))) +
                    casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].astype(int).astype(str) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_less_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x-1))) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_more_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x+1))) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_lib_exame'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['dt_diag'].apply(date_hash) )

In [65]:
casos_confirmados = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]

In [66]:
casos_confirmados.loc[casos_confirmados['uf_resid']!='PR']

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
5,13673,-1,SP,355030,410690,ITALO FERNANDO TROMBINI FILHO,M,28.0,Genoprimer,2020-03-16,2020-03-12,NaT,1,2020-06-17,2020-08-05,ITALOFERNANDOTROMBINIFILHO28.0355030,ITALOFERNANDOTROMBINIFILHO27.0355030,ITALOFERNANDOTROMBINIFILHO29.0355030,ITALOFERNANDOTROMBINIFILHO28.0410690,ITALOFERNANDOTROMBINIFILHO27.0410690,ITALOFERNANDOTROMBINIFILHO29.0410690,ITALOFERNANDOTROMBINIFILHO16032020
63,19531,-1,DF,530010,410690,LUCIANO PUCHALSKI,M,59.0,LACEN,2020-03-23,2020-03-24,2020-03-13,3,NaT,NaT,LUCIANOPUCHALSKI59.0530010,LUCIANOPUCHALSKI58.0530010,LUCIANOPUCHALSKI60.0530010,LUCIANOPUCHALSKI59.0410690,LUCIANOPUCHALSKI58.0410690,LUCIANOPUCHALSKI60.0410690,LUCIANOPUCHALSKI23032020
77,15204,-1,SP,355030,410370,JOAO HENRIQUE FONZAR BEGNINI,M,37.0,LACEN,2020-03-25,2020-03-25,2020-03-17,3,NaT,NaT,JOAOHENRIQUEFONZARBEGNINI37.0355030,JOAOHENRIQUEFONZARBEGNINI36.0355030,JOAOHENRIQUEFONZARBEGNINI38.0355030,JOAOHENRIQUEFONZARBEGNINI37.0410370,JOAOHENRIQUEFONZARBEGNINI36.0410370,JOAOHENRIQUEFONZARBEGNINI38.0410370,JOAOHENRIQUEFONZARBEGNINI25032020
175,19629,-1,RJ,330330,411435,LUCINEIA ALVES DE ANHAIA,F,38.0,LACEN,2020-03-31,2020-03-31,2020-03-22,3,NaT,NaT,LUCINEIAALVESDEANHAIA38.0330330,LUCINEIAALVESDEANHAIA37.0330330,LUCINEIAALVESDEANHAIA39.0330330,LUCINEIAALVESDEANHAIA38.0411435,LUCINEIAALVESDEANHAIA37.0411435,LUCINEIAALVESDEANHAIA39.0411435,LUCINEIAALVESDEANHAIA31032020
218,25622,-1,SP,355030,410690,PAULO FRANCISCO NETO,M,25.0,LACEN,2020-03-31,2020-04-01,2020-03-12,1,2020-03-19,2020-08-05,PAULOFRANCISCONETO25.0355030,PAULOFRANCISCONETO24.0355030,PAULOFRANCISCONETO26.0355030,PAULOFRANCISCONETO25.0410690,PAULOFRANCISCONETO24.0410690,PAULOFRANCISCONETO26.0410690,PAULOFRANCISCONETO31032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170199,1255819,-1,SC,421880,410940,VALDEMAR PINTO DE CRISTO,M,52.0,1,2021-06-10,2021-06-17,2021-06-09,3,NaT,NaT,VALDEMARPINTODECRISTO52.0421880,VALDEMARPINTODECRISTO51.0421880,VALDEMARPINTODECRISTO53.0421880,VALDEMARPINTODECRISTO52.0410940,VALDEMARPINTODECRISTO51.0410940,VALDEMARPINTODECRISTO53.0410940,VALDEMARPINTODECRISTO10062021
1170235,1260149,-1,SP,350070,411990,VALDIR APARECIDO DE OLIVEIRA,M,47.0,167,2021-05-31,2021-06-17,NaT,3,NaT,NaT,VALDIRAPARECIDODEOLIVEIRA47.0350070,VALDIRAPARECIDODEOLIVEIRA46.0350070,VALDIRAPARECIDODEOLIVEIRA48.0350070,VALDIRAPARECIDODEOLIVEIRA47.0411990,VALDIRAPARECIDODEOLIVEIRA46.0411990,VALDIRAPARECIDODEOLIVEIRA48.0411990,VALDIRAPARECIDODEOLIVEIRA31052021
1170248,1261229,-1,SC,420590,410940,VALDIRENE APARECIDA RIBAS,F,31.0,167,2021-06-13,2021-06-17,2021-06-11,3,NaT,NaT,VALDIRENEAPARECIDARIBAS31.0420590,VALDIRENEAPARECIDARIBAS30.0420590,VALDIRENEAPARECIDARIBAS32.0420590,VALDIRENEAPARECIDARIBAS31.0410940,VALDIRENEAPARECIDARIBAS30.0410940,VALDIRENEAPARECIDARIBAS32.0410940,VALDIRENEAPARECIDARIBAS13062021
1170438,1255529,-1,SP,354140,999999,VITOR ANTUNES PEREIRA,M,33.0,184,2021-06-12,2021-06-17,2021-06-09,3,NaT,NaT,VITORANTUNESPEREIRA33.0354140,VITORANTUNESPEREIRA32.0354140,VITORANTUNESPEREIRA34.0354140,VITORANTUNESPEREIRA33.0999999,VITORANTUNESPEREIRA32.0999999,VITORANTUNESPEREIRA34.0999999,VITORANTUNESPEREIRA12062021


In [68]:
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47.0,LACEN,2020-03-12,2020-03-12,NaT,1,2020-04-03,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47.0410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46.0410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48.0410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47.0410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46.0410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48.0410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43.0,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,FLAMARIONPETRIVDEABREU43.0410690,FLAMARIONPETRIVDEABREU42.0410690,FLAMARIONPETRIVDEABREU44.0410690,FLAMARIONPETRIVDEABREU43.0410690,FLAMARIONPETRIVDEABREU42.0410690,FLAMARIONPETRIVDEABREU44.0410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15.0,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15.0410690,FLAMARIONPETRIVDEABREUJUNIOR14.0410690,FLAMARIONPETRIVDEABREUJUNIOR16.0410690,FLAMARIONPETRIVDEABREUJUNIOR15.0410690,FLAMARIONPETRIVDEABREUJUNIOR14.0410690,FLAMARIONPETRIVDEABREUJUNIOR16.0410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26.0,Genoprimer,2020-03-16,2020-03-12,2020-03-11,1,NaT,NaT,GABRIELAFUGANTICASAGRANDE26.0410690,GABRIELAFUGANTICASAGRANDE25.0410690,GABRIELAFUGANTICASAGRANDE27.0410690,GABRIELAFUGANTICASAGRANDE26.0410690,GABRIELAFUGANTICASAGRANDE25.0410690,GABRIELAFUGANTICASAGRANDE27.0410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26.0,Genoprimer,2020-03-18,2020-03-12,2020-03-15,1,NaT,NaT,GABRIELASCHNEIDERGUGELMIN26.0410690,GABRIELASCHNEIDERGUGELMIN25.0410690,GABRIELASCHNEIDERGUGELMIN27.0410690,GABRIELASCHNEIDERGUGELMIN26.0410690,GABRIELASCHNEIDERGUGELMIN25.0410690,GABRIELASCHNEIDERGUGELMIN27.0410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170627,1257727,-1,PR,411580,411580,ZULMA ARLETE DA SILVEIRA,F,63.0,27,2021-05-17,2021-06-17,2021-05-11,1,2021-05-20,2021-06-17,ZULMAARLETEDASILVEIRA63.0411580,ZULMAARLETEDASILVEIRA62.0411580,ZULMAARLETEDASILVEIRA64.0411580,ZULMAARLETEDASILVEIRA63.0411580,ZULMAARLETEDASILVEIRA62.0411580,ZULMAARLETEDASILVEIRA64.0411580,ZULMAARLETEDASILVEIRA17052021
1170628,1261774,-1,PR,412570,412570,ZULMA BEATRIZ LOPEZ DE FERREIRA,F,49.0,27,2021-06-15,2021-06-17,2021-06-10,3,NaT,NaT,ZULMABEATRIZLOPEZDEFERREIRA49.0412570,ZULMABEATRIZLOPEZDEFERREIRA48.0412570,ZULMABEATRIZLOPEZDEFERREIRA50.0412570,ZULMABEATRIZLOPEZDEFERREIRA49.0412570,ZULMABEATRIZLOPEZDEFERREIRA48.0412570,ZULMABEATRIZLOPEZDEFERREIRA50.0412570,ZULMABEATRIZLOPEZDEFERREIRA15062021
1170629,1259702,-1,PR,410430,410430,ZULMIR LUIZ PASINI,M,76.0,1,2021-06-02,2021-06-17,2021-05-30,1,2021-06-14,2021-06-17,ZULMIRLUIZPASINI76.0410430,ZULMIRLUIZPASINI75.0410430,ZULMIRLUIZPASINI77.0410430,ZULMIRLUIZPASINI76.0410430,ZULMIRLUIZPASINI75.0410430,ZULMIRLUIZPASINI77.0410430,ZULMIRLUIZPASINI02062021
1170630,1258691,-1,PR,411300,410550,ZULMIRA APARECIDA MOREZZI GUIETI,F,69.0,167,2021-06-15,2021-06-17,2021-06-13,3,NaT,NaT,ZULMIRAAPARECIDAMOREZZIGUIETI69.0411300,ZULMIRAAPARECIDAMOREZZIGUIETI68.0411300,ZULMIRAAPARECIDAMOREZZIGUIETI70.0411300,ZULMIRAAPARECIDAMOREZZIGUIETI69.0410550,ZULMIRAAPARECIDAMOREZZIGUIETI68.0410550,ZULMIRAAPARECIDAMOREZZIGUIETI70.0410550,ZULMIRAAPARECIDAMOREZZIGUIETI15062021


In [69]:
cc = CasosConfirmados()
cc.df = casos_confirmados
cc.save('cc_19_06_2021',replace=True)

databases: ['cc_13_06_2021', 'cc_14_06_2021', 'cc_15_06_2021', 'cc_17_06_2021', 'cc_21_05_2021', 'cc_22_05_2021', 'cc_23_05_2021', 'cc_25_05_2021', 'cc_27_05_2021', 'cc_28_05_2021', 'cc_29_05_2021']
Load /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados/cc_19_06_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:02
